# **Web scraping**

* Scrape product names, descriptions and prices from a price comparison website
* Find highest, median and lowest competitor prices for each product to use as a benchmark
  ## 



In [743]:
# Crawl price comparison site to get list of product category pages
# Crawl category pages to get list of product pages

from bs4 import BeautifulSoup
import json
import pandas
import random
import re
import requests
import statistics


In [80]:
response = requests.get('https://www.pricerunner.com/')
soup = BeautifulSoup(response.text, 'lxml')
scrapt_tag = soup.find("script", id="initial_payload")

name_pattern1 = '"shortName":"Electronics"(.+?)"shortName"'
name_pattern2 = '"shortName":"Sound & TV"(.+?)"shortName"'
pattern = 'path(.+?),'

In [89]:
raw_data1 = re.findall(name_pattern1, scrapt_tag.string, re.S)
raw_data2 = re.findall(name_pattern2, scrapt_tag.string, re.S)
raw_data_both = raw_data1[0] + raw_data2[0]

data = re.findall(pattern, raw_data_both, re.S)
print(type(data), len(data))


<class 'list'> 320


In [201]:
url_list = []
if data:
    for i in data:
        if '.jpg' not in i and '?attr' not in i:
            k = i.strip('":"\\')
            n = k.split('u002F')
            j = ''
            for m in n:
                j += m.strip('\\') + '/'
                url = 'https://www.pricerunner.com' + j
            if '.com/t' not in url and 'Boat-' not in url and 'Musical-' not in url and 'Karaoke' not in url and 'DJ-' not in url and 'Party-' not in url:
                print(url)
                url_list.append(url)

https://www.pricerunner.com/cl/223/Desktop-Computers/
https://www.pricerunner.com/cl/27/Laptops/
https://www.pricerunner.com/cl/10012/Single-Board-Computers/
https://www.pricerunner.com/cl/224/Tablets/
https://www.pricerunner.com/cl/1156/eReaders/
https://www.pricerunner.com/cl/1426/3D-Printing/
https://www.pricerunner.com/cl/569/Computer-Accessories/
https://www.pricerunner.com/cl/110/Computer-Mice/
https://www.pricerunner.com/cl/182/Graphics-Tablets/
https://www.pricerunner.com/cl/187/Ink-Toners/
https://www.pricerunner.com/cl/111/Keyboards/
https://www.pricerunner.com/cl/10004/Laptop-Stands/
https://www.pricerunner.com/cl/25/Monitors/
https://www.pricerunner.com/cl/1579/Mouse-Pads/
https://www.pricerunner.com/cl/116/Printers/
https://www.pricerunner.com/cl/50/Scanners/
https://www.pricerunner.com/cl/49/Webcams/
https://www.pricerunner.com/cl/10005/External-Enclosures/
https://www.pricerunner.com/cl/36/Hard-Drives/
https://www.pricerunner.com/cl/233/Memory-Card-Readers/
https://www.p

In [195]:
data2 = []
for u in url_list:
    response = requests.get(u)
    soup = BeautifulSoup(response.text, 'lxml')
    scrapt_tag = soup.find("script", id="initial_payload")
    pattern2 = '"url"(.+?)",'
    data2.append(re.findall(pattern2, scrapt_tag.string, re.S))


In [197]:
print(type(data2), len(data2))

<class 'list'> 54


In [198]:
url_list_products = []
if data2:
    for i in data2:
        for h in i:
            if 'Compare-Prices' in h:
                k = h.strip('":"\\')
                n = k.split('u002F')
                j = ''
                for m in n:
                    j += m.strip('\\') + '/'
                    url = 'https://www.pricerunner.com' + j
                #print(url)
                url_list_products.append(url)
                
print(len(url_list_products), '\n')

print(url_list_products[:6])


3094 

['https://www.pricerunner.com/pl/223-3268585/Desktop-Computers/Vibox-Nemesis-Package-12-%28B00YHGDB4U%29-Compare-Prices/', 'https://www.pricerunner.com/pl/223-5252200/Desktop-Computers/Apple-iMac-%282020%29-3-1GHz-HC-8GB-256GB-27-Compare-Prices/', 'https://www.pricerunner.com/pl/223-5252210/Desktop-Computers/Apple-iMac-Retina-5K-Core-i5-3.3GHz-8GB-512GB-Radeon-Pro-5300-27-Compare-Prices/', 'https://www.pricerunner.com/pl/223-5315427/Desktop-Computers/MSI-Trident-X-Plus-9SE-614EU-Compare-Prices/', 'https://www.pricerunner.com/pl/223-3200009295/Desktop-Computers/Apple-Mac-mini-%282020%29-M1-8GB-256GB-SSD-Compare-Prices/', 'https://www.pricerunner.com/pl/223-3200034426/Desktop-Computers/Asrock-DeskMini-X300-%28Black%29-Compare-Prices/']


In [199]:
small_url_list = [url_list_products[1]]

small_url_list

['https://www.pricerunner.com/pl/223-5252200/Desktop-Computers/Apple-iMac-%282020%29-3-1GHz-HC-8GB-256GB-27-Compare-Prices/']

In [253]:
# crawl each url to take product_name, short_description, prices

data3 = []

In [499]:
for n in range(1, len(url_list_products)): 
    response = requests.get(url_list_products[n])
    soup = BeautifulSoup(response.text, 'lxml')
    scrapt_tag = soup.find("script", id="initial_payload")
    pattern_name = '"ogTitle"(.+?)",'
    pattern_descr = '"article"(.+?)",'
    pattern_prices = '"merchantId"(.+?)}'
    #pattern_links = '"url"(.+?)",'
    name = re.findall(pattern_name, scrapt_tag.string, re.S)
    descr = re.findall(pattern_descr, scrapt_tag.string, re.S)
    prices = re.findall(pattern_prices, scrapt_tag.string, re.S)
    #links = re.findall(pattern_links, scrapt_tag.string, re.S)
    data3.append((name, descr, prices))


In [500]:
print(type(data3), len(data3))

<class 'list'> 3008


In [501]:
# Remove duplicate products
data3_set = []
for i in data3:
    if i not in data3_set:
        data3_set.append(i)
print(type(data3_set), len(data3_set))

<class 'list'> 2569


In [569]:
# Remove products without descriptions
count = 0
data4_rough = []
for item in data3:
    prices = []
    for i in item[2]:
        temp_number = ''
        for c in i:
            if c.isnumeric() or c in ['{', '.']:
                temp_number += c
        prices_raw = temp_number.split('{')
        if len(prices_raw) == 2:
            prices.append(prices_raw)
    if len(item[1][0]) > 2:
        count += 1
        count_str = str(count)
        while len(count_str) < 6:
            count_str = '0' + count_str # give each product a unique ID
        data4_rough.append((count_str, item[0][0].lstrip(':"').rstrip('– Best deals on PriceRunner UK').rstrip('\\"'), item[1][0].lstrip(':"'), prices))
print(count)

643


In [570]:
data4_rough[0]

('000001',
 'Apple iMac (2020) - 3,1GHz HC 8GB 256GB 27',
 "This all-in-one computer from Apple works great for those who like gaming because if it's high-performance hardware. The Intel Core i5 processor in this computer is very high-end and the Radeon Pro 5300 graphics card is not made for advanced applications or games. The hard drive is the fastest one you can get in computers made for the general public.\\n\\n* RAM: 8 GB\\n* Grafikkortet is a Radeon Pro 5300\\n* Intel Core i5 processor\\n* 3 GHz processor speed\\n* 256 GB SSD drive\\n\\nThis model is equipped with a SSD drive, which is a quiet and faster hard drive compared to the traditional HDD drive. The hard drive has room for 256 GB of data. The internal memory (RAM) amounts to 8 GB. If you replace the RAM, take care to check that the RAM fits your motherboard.\\n\\nTo buy a new graphics card for your computer could be useful if you play a lot of graphics-heavy games. The graphics card that comes with this computer is a Radeo

In [544]:
# Function to find most common seller
def most_frequent(List):
    counter = 0
    num = List[0]
     
    for i in List:
        curr_frequency = List.count(i)
        if(curr_frequency> counter):
            counter = curr_frequency
            seller = i
    return seller

seller_ids =[]
for item in data4_rough:
    if item[3]:
        for i in range(len(item[3])):
            seller_ids.append(item[3][i][0])
            
most_frequent(seller_ids)

'10606'

In [690]:
# clean data to remove '\\n'
data_clean = []
count = 0
for item in data4_rough:
    descr = ''
    chunks = item[2].split('\\n')
    for chunk in chunks:
        if chunk:
            descr += ' ' + chunk.strip('*')
    seller_ids = []
    comp_prices = []
    min_price = ''
    max_price = ''
    med_price = ''
    own_prices = []
    for p_tup in item[3]:
        if p_tup[0] not in seller_ids:
            seller_ids.append(p_tup[0])
        if p_tup[0] == '10606':
            own_prices.append(p_tup[1])
        else:
            comp_prices.append(float(p_tup[1]))
    if '10606' in seller_ids:
        if len(comp_prices) >= 1:
            min_price = min(comp_prices)
        if len(comp_prices) >= 2:
            max_price = max(comp_prices)
        if len(comp_prices) >= 3:
            med_price = statistics.median(comp_prices)
        count += 1
        count_str = str(count)
        while len(count_str) < 6:
            count_str = '0' + count_str # give each product a unique ID
        #print((item[0], item[1], min(own_prices), min_price, med_price, max_price))
        data_clean.append((count_str, item[1], min(own_prices), descr.strip(), min_price, med_price, max_price))
            

In [700]:
for item in data_clean[45:50]:
    print(item[0], '"'+item[1]+'"', item[2], '"'+item[3][:30]+'"', item[4], item[5], item[6])

000046 "Apple iPad 10.9\" 64GB (2022)" 503.00 "Whether you want to read e-boo" 399.29 485.0 744.48
000047 "Apple iPad 10.9\" 256GB (2022)" 683.00 "This tablet is a very handy to" 528.99 793.995 1141.76
000048 "Amazon Fire HD 8 \"32GB 12th Generation (2022)" 69.99 "Whether you want to read e-boo" 99.99  99.99
000049 "Lenovo Tab M10 3rd Gen 10.1 64G" 146.99 "This tablet is a very handy to" 149.0 179.495 215.38
000050 "Apple iPad 256GB (2021)" 470.00 "If you find it's not ideal to " 387.66 520.825 767.0


In [749]:
# Extend data columns and format
data_extended = []
data_extended.append(('product_id', 'name', 'description', 'own_price', 'cost', 'profit_rate_%', 'min_price', 'med_price', 'max_price', 'in_stock', 'sales_old', 'sales_new', 'sales_change'))
for item in data_clean:
    item_2 = float(item[2])
    cost_factor = random.randint(110, 170)/100
    if item[4] and item[4] < item_2:
        cost = round(float(item[4]) / cost_factor, 2)
    else:
        cost = round(float(item[2]) / cost_factor, 2)
    profit_rate = round((item_2-cost)/cost*100, 1)
    in_stock = random.randint(1, 1000)
    sales_old = random.randint(1, 100)
    sales_new = random.randint(1, 100)
    change = sales_new - sales_old
    if item[4]:
        item_4 = round(float(item[4]), 2)
    else:
        item_4 = item[4]
    if item[5]:
        item_5 = round(float(item[5]), 2)
    else:
        item_5 = item[5]
    if item[6]:
        item_6 = round(float(item[6]), 2)
    else:
        item_6 = item[6]
    data_extended.append((str(item[0]), item[1], item[3], round(item_2, 2), cost, profit_rate, item_4, item_5, item_6, in_stock, sales_old, sales_new, change))
    

In [750]:
# Save as csv file
pd = pandas.DataFrame(data_extended)
pd.to_csv('project_1.2_data_products_prices.csv')

In [755]:
customer_products = []
for n in range(200):
    j = random.randint(1, len(data_extended)-1)
    k = random.randint(1, len(data_extended)-1)
    customer_products.append((data_extended[j][0], data_extended[k][0]))

In [758]:
# Save as csv file
pd2 = pandas.DataFrame(customer_products)
pd2.to_csv('project_1.1_customer_products.csv')